In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU, Lambda, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import tensorflow as tf
import random
import os

In [2]:
seed = 25
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# Load data
data = fetch_california_housing(as_frame=True)
df = data.frame
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [4]:
# Features and target
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# Feature Engineering
X['RoomsPerHousehold'] = X['AveRooms'] / X['AveOccup']
X['BedRoomRatio'] = X['AveBedrms'] / X['AveRooms']
X['PeoplePerHousehold'] = X['Population'] / X['AveOccup']
X['LatLong'] = X['Latitude'] * X['Longitude']
X['DistToLA'] = abs(X['Latitude'] - 34) + abs(X['Longitude'] + 118)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# No Lambda interaction
inputs = Input(shape=(X_train_scaled.shape[1],))
x = Dense(256)(inputs)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.3)(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.25)(x)

x = Dense(64)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.1)(x)
x = Dropout(0.2)(x)

x = Dense(32)(x)
x = LeakyReLU(0.1)(x)

output = Dense(1)(x)
model = Model(inputs=inputs, outputs=output)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 13)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,641 (190.00 KB)

 Trainable params: 47,745 (186.50 KB)

 Non-trainable params: 896 (3.50 KB)

In [6]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=Huber(),
    metrics=['mae']
)

early_stop = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=7, verbose=1, min_lr=1e-6
)

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.1,
    epochs=150,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5365 - mae: 0.8911 - val_loss: 0.3218 - val_mae: 0.6191 - learning_rate: 0.0010
Epoch 2/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2218 - mae: 0.5153 - val_loss: 0.1994 - val_mae: 0.4559 - learning_rate: 0.0010
Epoch 3/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1971 - mae: 0.4805 - val_loss: 0.1850 - val_mae: 0.4337 - learning_rate: 0.0010
Epoch 4/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1895 - mae: 0.4685 - val_loss: 0.1729 - val_mae: 0.4187 - learning_rate: 0.0010
Epoch 5/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1786 - mae: 0.4519 - val_loss: 0.1687 - val_mae: 0.4129 - learning_rate: 0.0010
Epoch 6/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1777 - mae: 0.4490 - val_loss: 0.1666 - val_mae: 0.4073 - learning_rate: 0.0010
Epoch 7/150
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1723 - mae: 0.4409 - val_loss: 0.1660 - val_mae: 0.4079 - learning_rate: 0.0010

In [7]:
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"📊 Test MAE: {mae:.4f}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - loss: 0.1315 - mae: 0.3630
📊 Test MAE: 0.3636


In [8]:
y_pred = model.predict(X_test_scaled).flatten()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 MAE: {mae:.4f}")
print(f"📉 RMSE: {rmse:.4f}")
print(f"📈 R² Score: {r2:.4f}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step
📊 MAE: 0.3636
📉 RMSE: 0.5522
📈 R² Score: 0.7673
